<a href="https://colab.research.google.com/github/21cs076/Real-Time-Landslide-Detection-and-Buried-Vehicle-Identification-using-YOLOv8/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install roboflow
!pip install kagglehub
!pip install onnx
!pip install onnxruntime
!pip install onnxruntime-gpu

from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov10n.pt')

# Perform inference on an image from Google Drive
results = model('https://ultralytics.com/images/bus.jpg')

# Iterate through the results and display each
for result in results:
    result.show()

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="4MptKsSsONT0fnQsIDc2")
project = rf.workspace("deslizamentos").project("landslide-detection-augmentation")
version = project.version(12)
dataset = version.download("yolov8")


In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv5 model (You can replace it with other YOLOv5 variants like yolov5m.pt, yolov5l.pt, etc.)
model = YOLO("yolov10n.pt")  # Replace with yolov5m.pt or yolov5l.pt for larger models

# Train the model on your Landslide dataset
model.train(
    data="/content/Landslide-Detection-Augmentation-12/data.yaml",  # Path to your data.yaml file
    epochs=150,  # Number of epochs (adjust based on your needs)
    batch=16,   # Batch size (adjust based on GPU memory)
    imgsz=640,  # Image size for training (640 is a good choice)
    project="runs/train",  # Directory to save training results
    name="landslide_detection",  # Name for the training run
    exist_ok=True  # Overwrite existing results if any
)

model.save("/content/Landslide-Detection-Augmentation-12/lsmodel.pt")

In [7]:
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("/content/Landslide-Detection-Augmentation-12/lsmodel.pt")  # Replace with your trained model path

# Path to the test dataset (folder of images)
test_images_path = "/content/Landslide-Detection-Augmentation-12/test/images"  # Replace with your test images path

# Use the model.val() method to evaluate the model on the test dataset
# The data.yaml file contains the information about the test set including the labels.
metrics = model.val(data="/content/Landslide-Detection-Augmentation-12/data.yaml", task="detect")

# Print the mAP
print(f"mAP: {metrics.box.maps}")

Ultralytics 8.3.56 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10n summary (fused): 285 layers, 2,695,196 parameters, 0 gradients, 8.2 GFLOPs


val: Scanning /content/Landslide-Detection-Augmentation-12/valid/labels.cache... 115 images, 37 backgrounds, 0 corrupt: 100%|██████████| 115/115 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.67it/s]


                   all        115        162      0.613      0.438      0.459      0.209
               Fissure         29         60      0.574      0.337       0.38      0.183
             Landslide         61        102      0.652      0.539      0.537      0.235
Speed: 0.3ms preprocess, 9.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val
mAP: [    0.18272     0.23505]


In [ ]:
import onnxruntime as ort
import numpy as np

# Load the exported ONNX model using onnxruntime for inference
onnx_model_path = '/root/.cache/kagglehub/datasets/pypiahmad/landslide4sense-challenge/versions/1/best_model.onnx'
onnx_session = ort.InferenceSession(onnx_model_path)

# Inference example using ONNX model (dummy input)
# Assuming input is a 640x640 image
dummy_input = np.random.rand(1, 3, 640, 640).astype(np.float32)

# Run inference
outputs = onnx_session.run(None, {'images': dummy_input})

# Print the outputs (you can process it based on your needs)
print(outputs)


In [ ]:
# Clone YOLOv8 repository
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
